In [106]:
import os    
from dotenv  import load_dotenv
from pathlib import Path
import requests
import pandas as pd
pd.options.display.max_colwidth = 600  # Set Full Display

import datetime
from dateutil import relativedelta
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta

import ast
# import json

In [157]:
suburb="Glen_Waverley"
suburb_search = "glen-waverley-vic-3150"


In [188]:

suburb="Mount_Waverley"
suburb_search = "mount-waverley-vic-3149"


# History Analysis:

### Initialization: Loading saved data

In [ ]:
# use r -- to avoid unicode error !
df_history_path=Path(f'Address_Code_History_Combined_{suburb}.xlsx')
df_history = pd.read_excel(df_history_path, parse_dates=True)

In [190]:
history_df = df_history[["address_check","recent_history"]]
print(history_df.info())
history_df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2190 entries, 0 to 2189
Data columns (total 2 columns):
address_check     2180 non-null object
recent_history    1874 non-null object
dtypes: object(2)
memory usage: 34.3+ KB
None


,address_check,recent_history
1249,4-sutton-ct-glen-waverley-vic-3150,"{'event': 'Rent', 'date': '2014-10-08', 'price': 350, 'agency': 'Harcourts - Judd White'}"
1281,35-dunscombe-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1997-06-26', 'price': 132000}"
631,5-triumph-cl-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2002-08-03', 'price': 518500}"
1683,81-view-mount-rd-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2004-02-26', 'price': 395000}"
175,unit-9-270-blackburn-rd-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2017-10-30', 'price': 475000, 'agency': 'Harcourts Wheelers Hill'}"


New 非常注意:

先排除掉 Null:

再排除 [ ]：

In [191]:
history_df2=history_df[history_df["recent_history"].notnull()]
history_df2B = history_df2[history_df2["recent_history"].str.contains("{")]
history_df2B

,address_check,recent_history
0,unit-1-22-melaleuca-dr-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2005-09-11', 'price': 391000, 'agency': 'Barry Plant Doherty Real Estate Glen Waverley'}"
1,unit-1-34-montclair-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2015-03-20', 'price': 0}"
2,10-wedge-ct-glen-waverley-vic-3150,"{'event': 'Rent', 'date': '2017-07-12', 'price': 500, 'agency': 'Ray White Real Estate Glen Waverley'}"
3,unit-4-47-nottingham-st-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2010-02-25', 'price': 620000, 'agency': 'Hocking Stuart Glen Waverley'}"
5,7-cooper-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2004-09-28', 'price': 312500}"
...,...,...
2185,117-whites-lane-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2016-11-02', 'price': 1150000, 'agency': 'Justin James'}"
2186,33-whites-lane-glen-waverley-vic-3150,"{'event': 'Rent', 'date': '2016-06-29', 'price': 550, 'agency': 'Ray White Real Estate Mount Waverley'}"
2187,25-kennedy-st-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2001-11-09', 'price': 398000}"
2188,27-ralton-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2016-04-27', 'price': 1220000, 'agency': 'Harcourts - Judd White'}"


篩選 Sold:

In [192]:
history_df3 = history_df2B[history_df2B["recent_history"].str.contains("Sold")]
history_df3 = history_df3.reset_index(drop=True)
history_df3

,address_check,recent_history
0,unit-1-22-melaleuca-dr-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2005-09-11', 'price': 391000, 'agency': 'Barry Plant Doherty Real Estate Glen Waverley'}"
1,unit-1-34-montclair-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2015-03-20', 'price': 0}"
2,unit-4-47-nottingham-st-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2010-02-25', 'price': 620000, 'agency': 'Hocking Stuart Glen Waverley'}"
3,7-cooper-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2004-09-28', 'price': 312500}"
4,33-willow-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1989-10-01', 'price': 110000}"
...,...,...
1434,698-highbury-rd-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1995-10-02', 'price': 13000}"
1435,706-highbury-rd-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1989-06-01', 'price': 185000}"
1436,117-whites-lane-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2016-11-02', 'price': 1150000, 'agency': 'Justin James'}"
1437,25-kennedy-st-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2001-11-09', 'price': 398000}"


提取日期：String

In [193]:
sold_date=[]
for i, row in history_df3.iterrows(): # print(i) # row number
    sold_date.append(ast.literal_eval(row["recent_history"])['date'])

sold_date_df=pd.DataFrame(sold_date)   
sold_date_df.columns = ["Date"]
sold_date_df = sold_date_df.reset_index(drop=True)
sold_date_df


,Date
0,2005-09-11
1,2015-03-20
2,2010-02-25
3,2004-09-28
4,1989-10-01
...,...
1434,1995-10-02
1435,1989-06-01
1436,2016-11-02
1437,2001-11-09


重組：

非常注意，兩個 df 必須 reset index -- 不然會 miss match !!

In [194]:
history_df4 = pd.concat([history_df3,sold_date_df],axis=1)
history_df4

,address_check,recent_history,Date
0,unit-1-22-melaleuca-dr-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2005-09-11', 'price': 391000, 'agency': 'Barry Plant Doherty Real Estate Glen Waverley'}",2005-09-11
1,unit-1-34-montclair-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2015-03-20', 'price': 0}",2015-03-20
2,unit-4-47-nottingham-st-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2010-02-25', 'price': 620000, 'agency': 'Hocking Stuart Glen Waverley'}",2010-02-25
3,7-cooper-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2004-09-28', 'price': 312500}",2004-09-28
4,33-willow-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1989-10-01', 'price': 110000}",1989-10-01
...,...,...,...
1434,698-highbury-rd-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1995-10-02', 'price': 13000}",1995-10-02
1435,706-highbury-rd-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1989-06-01', 'price': 185000}",1989-06-01
1436,117-whites-lane-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2016-11-02', 'price': 1150000, 'agency': 'Justin James'}",2016-11-02
1437,25-kennedy-st-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2001-11-09', 'price': 398000}",2001-11-09


轉換 Datatime:

In [195]:
history_df4["Date"] = pd.to_datetime(history_df4["Date"])
history_df4

,address_check,recent_history,Date
0,unit-1-22-melaleuca-dr-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2005-09-11', 'price': 391000, 'agency': 'Barry Plant Doherty Real Estate Glen Waverley'}",2005-09-11
1,unit-1-34-montclair-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2015-03-20', 'price': 0}",2015-03-20
2,unit-4-47-nottingham-st-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2010-02-25', 'price': 620000, 'agency': 'Hocking Stuart Glen Waverley'}",2010-02-25
3,7-cooper-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2004-09-28', 'price': 312500}",2004-09-28
4,33-willow-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1989-10-01', 'price': 110000}",1989-10-01
...,...,...,...
1434,698-highbury-rd-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1995-10-02', 'price': 13000}",1995-10-02
1435,706-highbury-rd-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1989-06-01', 'price': 185000}",1989-06-01
1436,117-whites-lane-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2016-11-02', 'price': 1150000, 'agency': 'Justin James'}",2016-11-02
1437,25-kennedy-st-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2001-11-09', 'price': 398000}",2001-11-09


In [196]:
# Check Date type:
history_df4.iloc[5]["Date"]

Timestamp('1987-01-01 00:00:00')

篩選，時間到現在間隔 5年以上的：

In [197]:
now_date = datetime.now()
now_date

datetime.datetime(2022, 4, 5, 2, 12, 36, 980430)

In [198]:
# 5年前：
fiveyrsago = now_date - relativedelta(years=5)
benckmark=datetime.strftime(fiveyrsago,"%Y-%m-%d")
# benckmark=datetime.strptime(benckmark,"%Y-%m-%d")
benckmark

'2017-04-05'

選擇 5 年以上的數據：

In [199]:
history_df5 = history_df4[history_df4["Date"]<benckmark]
history_df5 = history_df5.sort_values(by=["Date"],ascending = False)
history_df5

,address_check,recent_history,Date
1012,29-caithness-cres-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2017-04-03', 'price': 1668800, 'agency': 'Biggin & Scott Glen Waverley'}",2017-04-03
740,28-edith-st-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2017-04-01', 'price': 1716000, 'agency': 'Harcourts Glen Waverley'}",2017-04-01
857,22-montclair-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2017-03-25', 'price': 1565000, 'agency': 'Ray White Real Estate Glen Waverley'}",2017-03-25
151,18-ingleside-cres-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2017-03-25', 'price': 2388000, 'agency': 'Jrw Property International - Glen Waverley'}",2017-03-25
27,5-aurora-ct-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '2017-03-24', 'price': 0}",2017-03-24
...,...,...,...
744,28-fiander-ave-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1975-04-24', 'price': 28500}",1975-04-24
1148,15-rythdale-ct-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1975-04-14', 'price': 14950}",1975-04-14
868,8-devitt-ct-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1975-03-20', 'price': 39000}",1975-03-20
36,52-johnson-dr-glen-waverley-vic-3150,"{'event': 'Sold', 'date': '1975-03-13', 'price': 0}",1975-03-13


In [200]:
# create excel writer object
writer=pd.ExcelWriter(f'Analytics_History_Analysis_{suburb}.xlsx')
# write dataframe to excel
history_df5.to_excel(writer, index=False)
# save the excel##Ddaddddsddd
writer.save()
print('DataFrame is written successfully to Excel File.')

DataFrame is written successfully to Excel File.


# Suburb For Sale Properties: Days on the market

In [201]:
# use r -- to avoid unicode error !
filesaved_forsale_path=Path(rf'C:\Users\LEON\test\domain\Suburb_for_Sale_{suburb}.xlsx')
filesaved_forsale_df = pd.read_excel(filesaved_forsale_path, parse_dates=True)


In [202]:
for_sale_df = filesaved_forsale_df[["address","suburb_name","type","bed","bath","car","list_date","list_price","days_on_market"]]
print(for_sale_df.info())
for_sale_df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 9 columns):
address           141 non-null object
suburb_name       141 non-null object
type              141 non-null object
bed               141 non-null int64
bath              141 non-null int64
car               141 non-null int64
list_date         141 non-null object
list_price        57 non-null object
days_on_market    141 non-null int64
dtypes: int64(4), object(5)
memory usage: 10.0+ KB
None


,address,suburb_name,type,bed,bath,car,list_date,list_price,days_on_market
60,11 Emma Court,Mount Waverley,house,5,2,2,2022-03-29,"$1,500,000 - $1,650,000",7
87,1/33 Sesame Street,Mount Waverley,unit,3,2,2,2022-03-16,NaN,20
52,9/13-17 Forster Road,Mount Waverley,townhouse,3,2,2,2022-03-30,"$950,000 -$1,020,000",6
76,15 Sunhill Road,Mount Waverley,house,5,2,2,2022-03-23,"$1,450,000 - $1,550,000",13
6,1/26 Baringa Street,Mount Waverley,unit,2,1,1,2022-03-27,NaN,2


In [203]:
for_sale_df2=for_sale_df[for_sale_df["days_on_market"]>=0]
for_sale_df2=for_sale_df2.sort_values(by=["days_on_market"],ascending=True)
for_sale_df2

,address,suburb_name,type,bed,bath,car,list_date,list_price,days_on_market
0,4/19 Stanley Avenue,Mount Waverley,townhouse,4,3,2,2022-04-04,"$950,000 to $1,045,000",1
1,467 High Street Road,Mount Waverley,house,4,2,2,2022-04-04,NaN,1
2,2 St Andrews Court,Mount Waverley,house,4,1,5,2022-04-04,"$1,200,000 - $1,300,000",1
3,13 Oakern Street,Mount Waverley,house,3,2,2,2022-04-04,"$1,300,000 - $1,430,000",1
4,B&C/6 Glenarm Place,Mount Waverley,townhouse,4,3,2,2022-03-27,NaN,2
...,...,...,...,...,...,...,...,...,...
136,3 Bowman Street,Mount Waverley,house,4,3,2,2022-02-23,NaN,41
137,5/2-4 Doynton Pde,Mount Waverley,townhouse,3,4,2,2022-02-23,"$1,450,000",41
139,4/29-31 Lee Avenue,Mount Waverley,unit,2,1,1,2022-02-22,NaN,42
138,8/2-4 Doynton Pde,Mount Waverley,townhouse,4,3,2,2022-02-22,NaN,42


In [204]:
# create excel writer object
writer=pd.ExcelWriter(f'Analytics_For_Sale_Analysis_{suburb}.xlsx')
# write dataframe to excel
for_sale_df2.to_excel(writer, index=False)
# save the excel
writer.save()
print('DataFrame is written successfully to Excel File.')

DataFrame is written successfully to Excel File.


# Suburb Auction Results: 半年範圍

'Sold',

'Passed In',

'Private Sale',

'Withdrawn Prior To Auction',

In [205]:
# use r -- to avoid unicode error !
filesaved_auction_path=Path(rf'C:\Users\LEON\test\domain\Auction_Result_{suburb}.xlsx')
filesaved_auction_df = pd.read_excel(filesaved_auction_path, parse_dates=True)
print(filesaved_auction_df.info())
filesaved_auction_df.tail(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 13 columns):
address        148 non-null object
agency         148 non-null object
bed            148 non-null int64
id             148 non-null int64
is_sold        148 non-null bool
postcode       148 non-null int64
price          148 non-null int64
pu_id          110 non-null float64
result         148 non-null object
sold_date      148 non-null object
state          148 non-null object
suburb_name    148 non-null object
type           148 non-null object
dtypes: bool(1), float64(1), int64(4), object(7)
memory usage: 14.1+ KB
None


,address,agency,bed,id,is_sold,postcode,price,pu_id,result,sold_date,state,suburb_name,type
146,34 Meredith St,Barry Plant Monash,4,421991160,True,3149,1770000,3.973092e+10,Sold,2021-10-02,VIC,Mount Waverley,house
147,27 Lechte Rd,McGrath Waverley,4,421291160,False,3149,0,1.977408e+10,Passed In,2021-10-02,VIC,Mount Waverley,townhouse


In [206]:
auction_result_df = filesaved_auction_df[["address","type","bed","agency","is_sold","sold_date","price","result"]]
print(auction_result_df.info())
auction_result_df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 8 columns):
address      148 non-null object
type         148 non-null object
bed          148 non-null int64
agency       148 non-null object
is_sold      148 non-null bool
sold_date    148 non-null object
price        148 non-null int64
result       148 non-null object
dtypes: bool(1), int64(2), object(5)
memory usage: 8.4+ KB
None


,address,type,bed,agency,is_sold,sold_date,price,result
56,10/3 Keylana Bvd,townhouse,3,Barry Plant Monash,True,2022-01-29,830000,Private Sale
14,13 Bennett Av,house,3,Ray White Burwood,True,2022-03-26,0,"Sold Prior, Price Not Disclosed"
51,17 Hillside Rd,house,3,Harcourts First,True,2022-02-05,1690000,Sold
145,2/31 Adelaide Av,townhouse,3,McGrath Waverley,False,2021-10-02,0,Withdrawn Prior To Auction
46,2/17 Ian Gr,townhouse,4,Jellis Craig Monash,True,2022-02-12,1780000,Sold


非常注意：

一些房子先 pass in 但是後來賣掉！

In [207]:
# DataFrame.duplicated(subset=None, keep='first')

In [208]:
auction_result_df3 = auction_result_df[auction_result_df["address"].duplicated(keep=False)] #顯示全部重複的
auction_result_df3

,address,type,bed,agency,is_sold,sold_date,price,result
10,2/4 Prospect St,unit,3,Buxton Mount Waverley,True,2022-03-26,1095000,Private Sale
11,35 Lynden Gr,house,3,Ray White Blackburn,True,2022-03-26,1360000,Private Sale
20,2/45 Pamela St,townhouse,4,First National Real Estate Waverley City,True,2022-03-19,1220000,Private Sale
21,2/45 Pamela St,townhouse,4,First National Real Estate Waverley City,True,2022-03-12,1220000,Private Sale
29,35 Lynden Gr,house,3,Ray White Blackburn,False,2022-03-05,0,Passed In
37,34 Hillside Rd,house,4,Barry Plant Monash,True,2022-02-26,1585000,Sold
43,2/65 Wave Av,townhouse,3,The One Real Estate,True,2022-02-12,1122000,Private Sale
48,2/65 Wave Av,townhouse,3,The One Real Estate,True,2022-02-05,1122000,Private Sale
50,1/3 Fairway Av,townhouse,4,Area Specialist Keysborough,False,2022-02-05,0,Withdrawn Prior To Auction
52,10/3 Keylana Bvd,townhouse,3,Barry Plant Monash,True,2022-02-05,830000,Private Sale


非常注意：這個是所有重複地址的，且都賣掉的，即使之前沒賣掉。

In [209]:
auction_result_df4 = auction_result_df3[auction_result_df3["is_sold"]==True]
auction_result_df4

,address,type,bed,agency,is_sold,sold_date,price,result
10,2/4 Prospect St,unit,3,Buxton Mount Waverley,True,2022-03-26,1095000,Private Sale
11,35 Lynden Gr,house,3,Ray White Blackburn,True,2022-03-26,1360000,Private Sale
20,2/45 Pamela St,townhouse,4,First National Real Estate Waverley City,True,2022-03-19,1220000,Private Sale
21,2/45 Pamela St,townhouse,4,First National Real Estate Waverley City,True,2022-03-12,1220000,Private Sale
37,34 Hillside Rd,house,4,Barry Plant Monash,True,2022-02-26,1585000,Sold
43,2/65 Wave Av,townhouse,3,The One Real Estate,True,2022-02-12,1122000,Private Sale
48,2/65 Wave Av,townhouse,3,The One Real Estate,True,2022-02-05,1122000,Private Sale
52,10/3 Keylana Bvd,townhouse,3,Barry Plant Monash,True,2022-02-05,830000,Private Sale
55,4/644 Huntingdale Rd,unit,2,Ray White Oakleigh,True,2022-01-29,640000,Private Sale
56,10/3 Keylana Bvd,townhouse,3,Barry Plant Monash,True,2022-01-29,830000,Private Sale


In [210]:
auction_result_df2=auction_result_df[auction_result_df["is_sold"]!=True]
auction_result_df2

,address,type,bed,agency,is_sold,sold_date,price,result
28,15 Kay St,house,3,Fletchers Waverley,False,2022-03-05,0,Passed In
29,35 Lynden Gr,house,3,Ray White Blackburn,False,2022-03-05,0,Passed In
32,24A Oakern St,house,3,Buxton Mount Waverley,False,2022-03-05,0,Passed In
50,1/3 Fairway Av,townhouse,4,Area Specialist Keysborough,False,2022-02-05,0,Withdrawn Prior To Auction
57,2/4 Prospect St,unit,3,Buxton Mount Waverley,False,2021-12-18,0,Vendor Bid
66,539 Stephensons Rd,house,3,Buxton Mount Waverley,False,2021-12-18,0,Withdrawn Prior To Auction
69,47 Bruce St,house,5,McGrath Waverley,False,2021-12-18,0,Withdrawn Prior To Auction
79,9-11 Sesame St,house,5,Barry Plant Monash,False,2021-12-11,0,Passed In
80,12 Russell Cr,house,4,Barry Plant Monash,False,2021-12-11,0,Passed In
84,34 Hillside Rd,house,4,Barry Plant Monash,False,2021-12-11,0,Passed In


非常注意：如果沒賣掉table中含有，後來被賣掉的，那就篩除！

非常注意 ~ 是 NOT ！！

In [211]:
# ~auction_result_df2.address.isin(auction_result_df4.address)

In [212]:
auction_result_df5=auction_result_df2[~auction_result_df2.address.isin(auction_result_df4.address)]
auction_result_df5=auction_result_df5.sort_values(by=["address"],ascending=True)
auction_result_df5

,address,type,bed,agency,is_sold,sold_date,price,result
50,1/3 Fairway Av,townhouse,4,Area Specialist Keysborough,False,2022-02-05,0,Withdrawn Prior To Auction
119,1/3 Fairway Av,townhouse,4,Area Specialist Keysborough,False,2021-11-06,0,Passed In
129,1/8 Huxtable St,unit,2,Barry Plant Monash,False,2021-10-30,980000,Vendor Bid
80,12 Russell Cr,house,4,Barry Plant Monash,False,2021-12-11,0,Passed In
28,15 Kay St,house,3,Fletchers Waverley,False,2022-03-05,0,Passed In
145,2/31 Adelaide Av,townhouse,3,McGrath Waverley,False,2021-10-02,0,Withdrawn Prior To Auction
32,24A Oakern St,house,3,Buxton Mount Waverley,False,2022-03-05,0,Passed In
147,27 Lechte Rd,townhouse,4,McGrath Waverley,False,2021-10-02,0,Passed In
105,28&30 Howell Dr,house,6,Barry Plant Monash,False,2021-11-20,0,Vendor Bid
92,3/4 Edna St,unit,3,Barry Plant Monash,False,2021-12-04,0,Withdrawn Prior To Auction


In [213]:
# create excel writer object
writer=pd.ExcelWriter(f'Analytics_Auction_Analysis_{suburb}.xlsx')
# write dataframe to excel
auction_result_df5.to_excel(writer, index=False)
# save the excel
writer.save()
print('DataFrame is written successfully to Excel File.')

DataFrame is written successfully to Excel File.


# Suburb Sold Properties: 含 20 cherry, 一年範圍

## Initialization: Loading saved data

In [214]:
# use r -- to avoid unicode error !
filesaved_sold_path=Path(rf'C:\Users\LEON\test\domain\Sold_Result_{suburb}.xlsx')
filesaved_sold_df = pd.read_excel(filesaved_sold_path, parse_dates=True)
print(filesaved_sold_df.info())
filesaved_sold_df.tail(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722 entries, 0 to 721
Data columns (total 19 columns):
state                 722 non-null object
address               722 non-null object
suburb_code           722 non-null object
suburb_name           722 non-null object
postcode              722 non-null int64
image                 722 non-null object
type                  722 non-null object
bed                   722 non-null int64
bath                  722 non-null int64
car                   722 non-null int64
lat                   722 non-null float64
lng                   722 non-null float64
land                  513 non-null float64
sold_date             722 non-null object
sold_price            722 non-null int64
gnaf_pid              716 non-null object
pu_id                 722 non-null int64
agent_code            721 non-null object
sold_price_display    584 non-null object
dtypes: float64(3), int64(6), object(10)
memory usage: 107.3+ KB
None


,state,address,suburb_code,suburb_name,postcode,image,type,bed,bath,car,lat,lng,land,sold_date,sold_price,gnaf_pid,pu_id,agent_code,sold_price_display
721,VIC,2/3 Churcher Court,mount-waverley-vic-3149,Mount Waverley,3149,https://farm66.static.flickr.com/65535/51981657745_1457d97335.jpg,townhouse,4,2,2,-37.894041,145.109133,NaN,2022-04-03,1152000,GAVIC423064526,39785529351,VIC13820,"$1,122,000 - $1,176,000"


In [215]:
filesaved_sold_df[filesaved_sold_df["address"]=="3/4 Edna St"]

,state,address,suburb_code,suburb_name,postcode,image,type,bed,bath,car,lat,lng,land,sold_date,sold_price,gnaf_pid,pu_id,agent_code,sold_price_display


Inspection:

In [216]:
auction_result_df[auction_result_df["address"]=="3/4 Edna St"]

,address,type,bed,agency,is_sold,sold_date,price,result
92,3/4 Edna St,unit,3,Barry Plant Monash,False,2021-12-04,0,Withdrawn Prior To Auction


In [217]:
auction_result_df4[auction_result_df4["address"]=="1/3 Fairway Av"]

,address,type,bed,agency,is_sold,sold_date,price,result


In [218]:
auction_result_df[auction_result_df["address"]=="47 Bruce St"]

,address,type,bed,agency,is_sold,sold_date,price,result
69,47 Bruce St,house,5,McGrath Waverley,False,2021-12-18,0,Withdrawn Prior To Auction


PS:

非常注意：auction_result 更新不完整，有一些後來 sold, 沒有更新進去，也不在 SOLD PROPERTIES 裏面 ！！！！
